In [1]:
from Analyzer import *

In [2]:
ana = Analyzer()

In [3]:
ana.tree = uproot.open('data/tt.root')["Events"]   
ana.get_mask()
ana.init_events()

In [4]:
ana.copy_events_to_gpu()
ana.object_selection()
ana.event_selection()
ana.copy_events_from_gpu()
#ana.clear_devents()
ana.postprocess()
ana.eventsOut

{'nPassElectron': array([2, 1, 1, ..., 1, 1, 0], dtype=int32),
 'nPassMuon': array([0, 2, 1, ..., 1, 0, 0], dtype=int32)}

In [5]:
ana.store("s")
ana.clear_events()

In [ ]:
def compile_cuda_kernels(self):
    cufiles = ['./ExampleAnalyzer.cu']
    self.knl_mask = cu.elementwise.ElementwiseKernel(
        arguments="bool *mask, bool *HLT_Ele32_WPTight_Gsf, bool *HLT_IsoMu24, int *nElectron, int *nMuon",
        operation="mask[i] = (HLT_Ele32_WPTight_Gsf[i] || HLT_IsoMu24[i]) && (nElectron[i]>=2 || nMuon[i]>=2)" )

    # read code from src files
    code = self.codeMaskEventsStruct + self.codeEventsStruct
    for cufile in cufiles:
        f = open(cufile,"r") 
        code += f.read()
        f.close()

    # complie cuda code
    module = cu.compiler.SourceModule(  code )
    self.knl_objectSelection_electron = module.get_function('knl_objectSelection_electron')
    self.knl_objectSelection_muon     = module.get_function('knl_objectSelection_muon')